In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid", font_scale=1.5)
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import layers

df = pd.read_csv("C:\\Users\\Arjun\\Downloads\\Bitcoin.csv")

df = df[['Closing Price (USD)']]

from sklearn.preprocessing import StandardScaler
scale =  StandardScaler()
df['Closing Price (USD)']=scale.fit_transform(df[['Closing Price (USD)']])

hist = []
target = []
length = 30
for i in range(len(df['Closing Price (USD)'])-length):
    x = df['Closing Price (USD)'][i:i+length]
    y = df['Closing Price (USD)'][i+length]
    hist.append(x)
    target.append(y)

    
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

hist_scaled = hist.reshape((len(hist), length, 1))
#print(hist_scaled.shape)

X_train = hist_scaled[:2530,:,:]
X_test = hist_scaled[2530:,:,:]
y_train = target[:2530,:]
y_test = target[2530:,:]

newmodel = tf.keras.Sequential()
newmodel.add(layers.LSTM(units=64, return_sequences= True, input_shape=(X_train.shape[1],1)))
newmodel.add(layers.LSTM(units=64, return_sequences=True))
newmodel.add(layers.LSTM(units=64, return_sequences=True))
newmodel.add(layers.Dropout(0.2))
newmodel.add(layers.LSTM(units=64))
newmodel.add(layers.Dropout(0.2))
newmodel.add(layers.Dense(units=1))
#newmodel.summary()
newmodel.compile(optimizer='adam', loss='mean_squared_error')

history = newmodel.fit(X_train, y_train, epochs=20, batch_size=16)

pred = newmodel.predict(X_test)
pred = scale.inverse_transform(pred)
y_test = scale.inverse_transform(y_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real', linewidth=2)
plt.plot(pred, color='red', label='Prediction')
plt.title('Bitcoin Price Prediction')
plt.legend()
plt.show()

def predictor(model,X_train,n,length):
    x_input = X_train[-1]
    n_steps=length
    n_features=1
    temp_input=list(x_input)
    lst_output=[]
    i=0
    while(i<n):
        if(len(temp_input)>length):
            #print("new this")
            x_input=np.array(temp_input[1:])
              #print("{} day input {}".format(i,x_input.shape))
            #print(x_input)
            x_input = x_input.reshape((1, n_steps, n_features))
              #print(x_input.shape)
            #print(x_input)
            x_input=np.asarray(x_input).astype(np.float64)
            yhat = model.predict(x_input, verbose=0)
            #print("{} day output {}".format(i,yhat))
            temp_input.append(yhat)
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.append(yhat)
            i=i+1
            
        else:
            #print("inside else")
            x_input = x_input.reshape((1, n_steps, n_features))
              #print(x_input.shape)
            yhat = model.predict(x_input, verbose=0)
            #print("0 day output",yhat)
            temp_input.append(yhat)
            #print(len(temp_input))
            lst_output.append(yhat)
            i=i+1
    return lst_output


n = int(input("Which day's value you wanna know:"))

t = predictor(model=newmodel,X_train=X_train,n=n,length=length)

t =np.array(t)
t = t.reshape(n,1)

new_t = scale.inverse_transform(t)

new_t

ModuleNotFoundError: No module named 'tensorflow'